# Minimum Eigen Optimizer


Finding the solution to a Quadratic Unconstrained Binary Optimization(QUBO) problem is equivalent in finding the ground state of a corresponding Ising Hamiltonian - applicable to optimization, quantum chemistry and physics.
The binary variables {0,1} are replaced by spin variables with values {−1,+1}, which allows to replace the resulting spin variables by Pauli Z matrices, and thus, an Ising Hamiltonian. 
Details on this mapping -> [1].

Qiskit provides automatic conversion from a suitable QuadraticProgram to an Ising Hamiltonian, which then allows to leverage all the MinimumEigenSolver such as - VQE, - QAOA, or - NumpyMinimumEigensolver(classical exact method).

Qiskit wraps the translation to an Ising Hamiltonian (in Qiskit Aqua also called Operator), the call to an MinimumEigensolver as well as the translation of the results back to OptimizationResult in the MinimumEigenOptimizer.

In the following we first illustrate the conversion from a QuadraticProgram to an Operator and then show how to use the MinimumEigenOptimizer with different MinimumEigensolver to solve a given QuadraticProgram. The algorithms in Qiskit automatically try to convert a given problem to the supported problem class if possible, for instance, the MinimumEigenOptimizer will automatically translate integer variables to binary variables or add a linear equality constraints as a quadratic penalty term to the objective. It should be mentioned that Aqua will through a QiskitOptimizationError if conversion of a quadratic program with integer variable is attempted.

The circuit depth of QAOA potentially has to be increased with the problem size, which might be prohibitive for near-term quantum devices. A possible workaround is Recursive QAOA, as introduced in [2]. Qiskit generalizes this concept to the RecursiveMinimumEigenOptimizer, which is introduced at the end of this tutorial.
References

[1] [A. Lucas, Ising formulations of many NP problems, Front. Phys., 12 (2014).](https://arxiv.org/abs/1302.5843)

[2] [S. Bravyi, A. Kliesch, R. Koenig, E. Tang, Obstacles to State Preparation and Variational Optimization from Symmetry Protection, arXiv preprint arXiv:1910.08980 (2019).](https://arxiv.org/abs/1910.08980)


## Converting a QUBO to an Operator

In [2]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

In [3]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var('x01')
qubo.binary_var('x02')
qubo.binary_var('x03')
qubo.binary_var('x04')
qubo.binary_var('x05')
qubo.binary_var('x06')
qubo.binary_var('x07')
qubo.binary_var('x08')
qubo.binary_var('x09')
qubo.binary_var('x10')
qubo.binary_var('x11')
qubo.binary_var('x12')
qubo.binary_var('x13')
qubo.binary_var('x14')
qubo.binary_var('x15')
qubo.binary_var('x16')
qubo.binary_var('x21')
qubo.binary_var('x22')
qubo.binary_var('x23')
qubo.binary_var('x24')
qubo.binary_var('x25')
qubo.binary_var('x26')
qubo.binary_var('x27')
qubo.binary_var('x28')
qubo.binary_var('x29')
qubo.binary_var('x30')
qubo.binary_var('x31')
qubo.binary_var('x32')
qubo.binary_var('x33')
qubo.binary_var('x34')
qubo.binary_var('x35')
qubo.binary_var('x36')
qubo.minimize(linear=[1,-2,3,4,5,-6,7,8,9,10,-11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32], 
              quadratic={('x01', 'x03'): 1, ('x11', 'x12'): -1, ('x14', 'x15'): 2})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x01 - 2 x02 + 3 x03 + 4 x04 + 5 x05 - 6 x06 + 7 x07 + 8 x08 + 9 x09
      + 10 x10 - 11 x11 + 12 x12 + 13 x13 + 14 x14 + 15 x15 + 16 x16 + 17 x21
      + 18 x22 + 19 x23 + 20 x24 + 21 x25 + 22 x26 + 23 x27 + 24 x28 + 25 x29
      + 26 x30 + 27 x31 + 28 x32 + 29 x33 + 30 x34 + 31 x35 + 32 x36 + [
      2 x01*x03 - 2 x11*x12 + 4 x14*x15 ]/2
Subject To

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x03 <= 1
 0 <= x04 <= 1
 0 <= x05 <= 1
 0 <= x06 <= 1
 0 <= x07 <= 1
 0 <= x08 <= 1
 0 <= x09 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1
 0 <= x12 <= 1
 0 <= x13 <= 1
 0 <= x14 <= 1
 0 <= x15 <= 1
 0 <= x16 <= 1
 0 <= x21 <= 1
 0 <= x22 <= 1
 0 <= x23 <= 1
 0 <= x24 <= 1
 0 <= x25 <= 1
 0 <= x26 <= 1
 0 <= x27 <= 1
 0 <= x28 <= 1
 0 <= x29 <= 1
 0 <= x30 <= 1
 0 <= x31 <= 1
 0 <= x32 <= 1
 0 <= x33 <= 1
 0 <= x34 <= 1
 0 <= x35 <= 1
 0 <= x36 <= 1

Binaries
 x01 x02 x03 x04 x05 x06 x07 x08 x09 x10 x11 x12 

In [4]:
#Next we translate this QUBO into an Ising operator. 
#This results not only in an Operator but also in a constant offset to be taking into account 
#to shift the resulting value.

op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 245.5
operator:
SummedOp([
  -0.75 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ,
  IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI,
  -1.75 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII,
  -2.0 * IIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII,
  -2.5 * IIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII,
  3.0 * IIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII,
  -3.5 * IIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII,
  -4.0 * IIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII,
  -4.5 * IIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII,
  -5.0 * IIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII,
  5.75 * IIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII,
  -5.75 * IIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII,
  -6.5 * IIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII,
  -7.5 * IIIIIIIIIIIIIIIIIIZIIIIIIIIIIIII,
  -8.0 * IIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII,
  -8.0 * IIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII,
  -8.5 * IIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII,
  -9.0 * IIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII,
  -9.5 * IIIIIIIIIIIIIZIIIIIIIIIIIIIIIIII,
  -10.0 * IIIIIIIIIIIIZIIIIIIIIIIIIIIIIIII,
  -10.5 * IIIIIIIIIIIZIIIIIIIIIIIIIIIIIIII,
  -11.0 * IIIIIIIIIIZIIIIIIIIIIIIIIIIIIIII,
  -11.5 * IIIIIIIIIZI

In [5]:
#Sometimes a QuadraticProgram might also directly be given in the form of an Operator. 
#For such cases, Qiskit also provides a converter from an Operator back to a QuadraticProgram, 
#which we illustrate in the following.

qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x_0 - 2 x_1 + 3 x_2 + 4 x_3 + 5 x_4 - 6 x_5 + 7 x_6 + 8 x_7 + 9 x_8
      + 10 x_9 - 11 x_10 + 12 x_11 + 13 x_12 + 14 x_13 + 15 x_14 + 16 x_15
      + 17 x_16 + 18 x_17 + 19 x_18 + 20 x_19 + 21 x_20 + 22 x_21 + 23 x_22
      + 24 x_23 + 25 x_24 + 26 x_25 + 27 x_26 + 28 x_27 + 29 x_28 + 30 x_29
      + 31 x_30 + 32 x_31 + [ 2 x_0*x_2 - 2 x_10*x_11 + 4 x_13*x_14 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1
 0 <= x_10 <= 1
 0 <= x_11 <= 1
 0 <= x_12 <= 1
 0 <= x_13 <= 1
 0 <= x_14 <= 1
 0 <= x_15 <= 1
 0 <= x_16 <= 1
 0 <= x_17 <= 1
 0 <= x_18 <= 1
 0 <= x_19 <= 1
 0 <= x_20 <= 1
 0 <= x_21 <= 1
 0 <= x_22 <= 1
 0 <= x_23 <= 1
 0 <= x_24 <= 1
 0 <= x_25 <= 1
 0 <= x_26 <= 1
 0 <= x_27 <= 1
 0 <= x_28 <= 1
 0 <= x_29 <= 1
 0 <= x_30 <= 1
 0 <= x_31 <= 1

Binaries
 

In [6]:
#This converter allows to translate an Operator to a QuadraticProgram and 
#then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, 
#such as the GroverOptimizer.

## Solving a QUBO with the MinimumEigenOptimizer

In [7]:
#We start by initializing the MinimumEigensolver we want to use.
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('qasm_simulator'))
exact_mes = NumPyMinimumEigensolver()

#Then, we use the MinimumEigensolver to create MinimumEigenOptimizer.
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [8]:
#We first use the MinimumEigenOptimizer based on the classical exact 
#NumPyMinimumEigensolver to get the optimal benchmark solution for this small example.
exact_result = exact.solve(qubo)
print(exact_result)

MemoryError: Unable to allocate 16.0 GiB for an array with shape (4294967297,) and data type uint32

In [9]:
#Next we apply the MinimumEigenOptimizer based on QAOA to the same problem.
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

BasicAerError: 'Number of qubits 32 is greater than maximum (24) for "qasm_simulator".'

## RecursiveMinimumEigenOptimizer

In [10]:
#The RecursiveMinimumEigenOptimizer takes a MinimumEigenOptimizer as input and 
#applies the recursive optimization scheme to reduce the size of the problem one variable at a time. 
#Once the size of the generated intermediate problem is below a given threshold (min_num_vars), 
#the RecursiveMinimumEigenOptimizer uses another solver (min_num_vars_optimizer), 
#e.g., an exact classical solver such as CPLEX or the MinimumEigenOptimizer based on the NumPyMinimumEigensolver.

#In the following, we show how to use the RecursiveMinimumEigenOptimizer 
#using the two MinimumEigenOptimizer introduced before.

#First, we construct the RecursiveMinimumEigenOptimizer 
#such that it reduces the problem size from 3 variables to 1 variable and 
#then uses the exact solver for the last variable. 
#Then we call solve to optimize the considered problem.

rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=5, min_num_vars_optimizer=exact)

rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)



BasicAerError: 'Number of qubits 32 is greater than maximum (24) for "qasm_simulator".'